### Allen download
##### The following function downloads the tracer-data for the WT-experiments and exports the data as a nifti file. In addition it also creates a cvs file with the experiment ids, which is needed for the allen function. It accepts the following arguments:
- resolution: the resolution of the tracer-data that is to be downloaded (default=100)
- directory: the directory in which the data and the experiment_id list is exported   (default = current directory
- brain_mask: speficies whether the data outside of the brain should be converted to zero's. 
 (default=true) 

##### If you just run the function without specifying anything (allen_download()). The function will simply proceed to download the tracer data at 100 muM resolution with a brain mask, and export the data, togehter with with csv-file to the current working directory. 

In [5]:
def allen_download (resolution=100, directory=None, brain_mask=True):
    
    #getting some packages
    import pandas as pd
    import numpy as np
    import os
    import nibabel as nib
    from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
    mcc = MouseConnectivityCache(resolution=resolution)
    
    #setting the directory
    if directory!=None:
        os.chdir(directory)
        
    #creating the projection search file
    id_list=mcc.get_experiments(cre=False)
    id_list=pd.DataFrame(id_list)
    id_list.to_csv('WT_experiment_id_list.csv', index=False)
    id_list=id_list['id'].to_list()

    #getting one tracer map's size
    experiment_id = 180436360
    single_tracer= mcc.get_projection_density(experiment_id)


    #creating the empty 4d_matrix
    x, y , z = single_tracer[0].shape
    i=len(id_list)
    merge_matrix = np.zeros(x*y*z*i)
    merge_matrix = np.reshape(merge_matrix,(x,y,z,i))

    if brain_mask==True:
        #getting the template
        template, template_info = mcc.get_template_volume()

    #saving the tracer values in the empty matrix
    a=0
    for tracer_id in id_list:
        
        tracer= (mcc.get_projection_density(tracer_id)[0])
        if brain_mask==True:
            tracer[template<=2]=0#mask for cortex
        merge_matrix[:,:,:,a]=tracer
        a=a+1
    if brain_mask==True:
        tracer_nifti=nib.Nifti1Image(merge_matrix, affine=np.eye(4))
        nib.save(tracer_nifti, 'tracer_data_masked.nii.gz')
    if brain_mask==False:
        tracer_nifti=nib.Nifti1Image(merge_matrix, affine=np.eye(4))
        nib.save(tracer_nifti, 'tracer_data.nii.gz')
  

In [6]:
#example code
allen_download()